In [ ]:
# NeurIPS Open Polymer Prediction 2025 - Baseline Model
# Kaggle環境用ベースラインモデル（基本的なML手法）

# 依存関係インストール（Kaggle環境では通常不要だが念のため）
import subprocess
import sys

def install_if_needed(package_name, import_name=None):
    """必要に応じてパッケージをインストール"""
    if import_name is None:
        import_name = package_name
    
    try:
        __import__(import_name)
        print(f"✅ {package_name} は既にインストール済み")
    except ImportError:
        print(f"📦 {package_name} をインストール中...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name, "--quiet"])
        print(f"✅ {package_name} インストール完了")

# 基本パッケージのインストール確認
install_if_needed("xgboost")

import numpy as np
import pandas as pd

# 入力ディレクトリの確認
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# 基本的なライブラリのインポート（ベースライン用）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
import time
import warnings
warnings.filterwarnings('ignore')

print("ベースライン用ライブラリインポート完了")

In [ ]:
# データ読み込みと基本分析（ベースライン版）
print(f"ベースラインポリマー特性予測パイプライン開始...")
print(f"実行日時: 2025-06-24")
start_time = time.time()

# 再現性のためのランダムシード設定
SEED = 42
np.random.seed(SEED)

# データ読み込み（Kaggle環境用パス）
train = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
submission = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')

print(f"訓練データ形状: {train.shape}")
print(f"テストデータ形状: {test.shape}")

# 欠損値確認
print("\n訓練データの欠損値:")
missing_values = train.isnull().sum()
print(missing_values)

# ターゲット列
target_cols = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']

# 各特性の利用可能サンプル数計算
available_samples = {col: train.shape[0] - missing_values[col] for col in target_cols}
print("\n各特性の利用可能サンプル数:")
for col, count in available_samples.items():
    print(f"{col}: {count} ({count/train.shape[0]*100:.2f}%)")

# ターゲット特性の統計情報
print("\nターゲット特性統計:")
print(train[target_cols].describe())

In [ ]:
# 基本的な特徴量抽出関数（ベースライン版）
def create_basic_features(df):
    """SMILES文字列から基本的な特徴量を作成（ベースライン版）"""
    features = pd.DataFrame()
    
    # 文字列長
    features['smiles_length'] = df['SMILES'].str.len()
    
    # 基本的な原子カウント
    features['carbon_count'] = df['SMILES'].str.count('C')
    features['nitrogen_count'] = df['SMILES'].str.count('N') 
    features['oxygen_count'] = df['SMILES'].str.count('O')
    features['sulfur_count'] = df['SMILES'].str.count('S')
    features['fluorine_count'] = df['SMILES'].str.count('F')
    
    # 結合情報
    features['single_bond_count'] = df['SMILES'].str.count('-')
    features['double_bond_count'] = df['SMILES'].str.count('=')
    features['triple_bond_count'] = df['SMILES'].str.count('#')
    
    # 環構造
    features['ring_count'] = df['SMILES'].str.count('1') + df['SMILES'].str.count('2')
    features['aromatic_count'] = df['SMILES'].str.count('c')
    
    # 分岐構造
    features['branch_count'] = df['SMILES'].str.count('(')
    
    # 基本比率
    features['carbon_ratio'] = features['carbon_count'] / features['smiles_length']
    features['hetero_ratio'] = (features['nitrogen_count'] + features['oxygen_count'] + features['sulfur_count']) / features['smiles_length']
    
    return features

print("基本特徴量抽出関数定義完了（ベースライン版）")

In [ ]:
# 基本特徴量生成（ベースライン版）
print("\n基本特徴量生成中...")

# 訓練データとテストデータの特徴量生成
train_features = create_basic_features(train)
test_features = create_basic_features(test)

print(f"特徴量生成完了:")
print(f"- 訓練データ特徴量: {train_features.shape}")
print(f"- テストデータ特徴量: {test_features.shape}")
print(f"- 特徴量数: {train_features.shape[1]}")

# 特徴量一覧表示
print(f"\n特徴量一覧: {list(train_features.columns)}")

# 基本統計
print("\n特徴量基本統計:")
print(train_features.describe())

In [ ]:
# データ準備とスケーリング
print("\nデータ準備中...")

# 特徴量をnumpy arrayに変換
X_train = train_features.values
X_test = test_features.values

# 欠損値を0で補完（基本的な手法）
X_train = np.nan_to_num(X_train, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)

# 標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"データ準備完了:")
print(f"- 訓練データ: {X_train_scaled.shape}")
print(f"- テストデータ: {X_test_scaled.shape}")

In [ ]:
# ベースラインモデル定義
def train_baseline_model(X, y, target_name):
    """ベースラインモデルの訓練（シンプルなXGBoost）"""
    
    # 欠損値のない行にフィルタ
    valid_idx = ~np.isnan(y)
    X_valid = X[valid_idx]
    y_valid = y[valid_idx]
    
    if len(y_valid) < 10:
        print(f"⚠️ {target_name}: データ不足 ({len(y_valid)} サンプル)")
        return None, 0.0
    
    print(f"🔄 {target_name} モデル訓練中... ({len(y_valid)} サンプル)")
    
    # 5-Fold CV
    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
    cv_scores = []
    models = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_valid)):
        X_train_fold, X_val_fold = X_valid[train_idx], X_valid[val_idx]
        y_train_fold, y_val_fold = y_valid[train_idx], y_valid[val_idx]
        
        # シンプルなXGBoostモデル
        model = xgb.XGBRegressor(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            random_state=SEED+fold,
            verbosity=0
        )
        model.fit(X_train_fold, y_train_fold)
        
        # 検証
        val_pred = model.predict(X_val_fold)
        score = mean_absolute_error(y_val_fold, val_pred)
        cv_scores.append(score)
        models.append(model)
        
        print(f"  Fold {fold+1}: MAE = {score:.4f}")
    
    avg_score = np.mean(cv_scores)
    print(f"✅ {target_name} CV MAE: {avg_score:.4f}")
    
    return models, avg_score

print("ベースラインモデル定義完了")

In [ ]:
# 全ターゲット特性のモデル訓練（ベースライン）
print("🚀 ベースラインモデル訓練開始")

# 各ターゲットのモデル訓練
trained_models = {}
cv_scores = {}

for target_col in target_cols:
    y = train[target_col].values
    models, score = train_baseline_model(X_train_scaled, y, target_col)
    trained_models[target_col] = models
    cv_scores[target_col] = score

print("\n📊 クロスバリデーション結果:")
for target_col, score in cv_scores.items():
    print(f"{target_col}: {score:.4f}")

overall_score = np.mean([s for s in cv_scores.values() if s > 0])
print(f"\n🎯 全体平均MAE: {overall_score:.4f}")

print("\nベースラインモデル訓練完了")

In [ ]:
# テスト予測とフォールバック処理
print("🔮 テスト予測中...")

test_predictions = {}

for target_col in target_cols:
    if trained_models[target_col] is not None:
        # 各フォールドの予測を平均
        fold_preds = []
        for model in trained_models[target_col]:
            pred = model.predict(X_test_scaled)
            fold_preds.append(pred)
        
        test_predictions[target_col] = np.mean(fold_preds, axis=0)
        print(f"✅ {target_col} 予測完了")
    else:
        # データ不足の場合は訓練データの平均値を使用
        mean_val = train[target_col].mean()
        if np.isnan(mean_val):
            # 完全に欠損している場合のフォールバック値
            fallback_values = {
                'Tg': 400,
                'FFV': 0.2,
                'Tc': 0.2,
                'Density': 1.0,
                'Rg': 10.0
            }
            mean_val = fallback_values.get(target_col, 0.0)
        
        test_predictions[target_col] = np.full(len(test), mean_val)
        print(f"⚠️ {target_col} 平均値で予測 (値: {mean_val:.3f})")

print("テスト予測完了")

In [ ]:
# 提出ファイル作成
print("📄 提出ファイル作成中...")

# 提出データフレーム作成
submission_final = pd.DataFrame({'id': test['id']})
for target_col in target_cols:
    submission_final[target_col] = test_predictions[target_col]

# 提出フォーマット確認
print("提出ファイルプレビュー:")
print(submission_final.head())

print(f"\n提出ファイル統計:")
for target_col in target_cols:
    values = submission_final[target_col]
    print(f"{target_col}: 平均={values.mean():.3f}, 標準偏差={values.std():.3f}")

# 提出ファイル保存
submission_final.to_csv('submission.csv', index=False)
print("\n✅ 提出ファイル保存完了: submission.csv")

elapsed_time = time.time() - start_time
print(f"\n⏱️ 総実行時間: {elapsed_time/60:.2f} 分")

print("\n🎉 ベースラインモデル実行完了！")